In [1]:
import torch
from torch import nn,optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.datasets import MNIST

In [2]:
class cnn_classify(nn.Module):                         
    def __init__(self):                                       
        super(cnn_classify, self).__init__()
        
        self.features = nn.Sequential(nn.Conv2d(1, 16, 3),      
                                      nn.ReLU(),                
                                      nn.MaxPool2d(2, 2),       
                                      nn.Conv2d(16, 10, 3),     
                                      nn.ReLU(),                
                                      nn.MaxPool2d(2))          
        
        self.classifier = nn.Sequential(nn.Linear(250, 128), 
                                        nn.ReLU(),
                                        nn.Dropout(p=0.4),
                                        nn.Linear(128, 84), 
                                        nn.ReLU(),
                                        nn.Dropout(p=0.4),
                                        nn.Linear(84, 10),
                                        nn.LogSoftmax(dim=1))
                                              
        
    def forward(self, image, prints=False):                     
        if prints: print('Original Image shape:', image.shape)
        
        
        image = self.features(image)
        if prints: print('Convol Image shape:', image.shape)
        
        
        image = image.view(-1, 250)
        if prints: print('Vectorized Image shape:', image.shape)
        
        
        out = self.classifier(image)
        if prints: print('Out:', out)
            
        
        return out

In [3]:
model = cnn_classify()
model

cnn_classify(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 10, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=250, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=128, out_features=84, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=84, out_features=10, bias=True)
    (7): LogSoftmax(dim=1)
  )
)

In [4]:
mnist_train = MNIST('data', train=True, download=True, transform=transforms.ToTensor())
mnist_valid = MNIST('data', train=False, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(mnist_valid, batch_size=64)



In [5]:
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [6]:
model = model.to(device)
epochs = 3
steps = 0
running_loss = 0
print_every = 100
for epoch in range(epochs):
    for inputs, labels in iter(trainloader):
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)

                    test_loss += batch_loss.item()


                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Validation loss: {test_loss/len(validloader):.3f}.. "
                      f"Validation accuracy: {accuracy/len(validloader):.3f}")
                running_loss = 0
                model.train()


Epoch 1/3.. Train loss: 1.666.. Validation loss: 0.784.. Validation accuracy: 0.757
Epoch 1/3.. Train loss: 0.805.. Validation loss: 0.458.. Validation accuracy: 0.864
Epoch 1/3.. Train loss: 0.601.. Validation loss: 0.393.. Validation accuracy: 0.884
Epoch 1/3.. Train loss: 0.541.. Validation loss: 0.299.. Validation accuracy: 0.910
Epoch 1/3.. Train loss: 0.425.. Validation loss: 0.254.. Validation accuracy: 0.919
Epoch 1/3.. Train loss: 0.419.. Validation loss: 0.232.. Validation accuracy: 0.928
Epoch 1/3.. Train loss: 0.349.. Validation loss: 0.228.. Validation accuracy: 0.928
Epoch 1/3.. Train loss: 0.378.. Validation loss: 0.238.. Validation accuracy: 0.927
Epoch 1/3.. Train loss: 0.343.. Validation loss: 0.199.. Validation accuracy: 0.936
Epoch 2/3.. Train loss: 0.329.. Validation loss: 0.199.. Validation accuracy: 0.937
Epoch 2/3.. Train loss: 0.307.. Validation loss: 0.187.. Validation accuracy: 0.940
Epoch 2/3.. Train loss: 0.329.. Validation loss: 0.192.. Validation accuracy